In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train_july=pd.read_csv("data/train_July.csv")
train_aug=pd.read_csv("data/train_Aug.csv")
weather=pd.read_csv("data/weather.csv")
poi=pd.read_csv("data/poi.csv")
test=pd.read_csv("data/test_id_Aug_agg_public5k.csv")

In [3]:
#划分测试集和训练集
train_july['day']=train_july['create_date'].apply(lambda x:int(str(x).split('-')[2]))
train_aug['day']=train_aug['create_date'].apply(lambda x:int(str(x).split('-')[2]))
test['day']=test['create_date'].apply(lambda x:int(str(x).split('-')[2]))
train1=train_july[train_july['create_date']<'2017-07-25']
train2=train_july[train_july['create_date']>='2017-07-25']
test1=train2[(train2['day']%2==0)&(train2['create_hour']%2==0)|(train2['day']%2==1)&(train2['create_hour']%2==1)]
train2=train2[(train2['day']%2==1)&(train2['create_hour']%2==0)|(train2['day']%2==0)&(train2['create_hour']%2==1)]

In [4]:
data_dict=test1.groupby(['start_geo_id','end_geo_id','create_date','create_hour'])['id'].count().to_dict()

In [5]:
test1['count']=test1.apply(lambda x:data_dict[tuple([x['start_geo_id'],x['end_geo_id'],x['create_date'],x['create_hour']])],axis=1)

In [6]:
test1=test1[test1[['start_geo_id','end_geo_id','create_date','create_hour']].duplicated()==False]
test1['id']=range(0,len(test1.values))

In [7]:
X_data=test1[['id','count']]

In [8]:
test1=test1.drop('count',axis=1)

In [9]:
def evaluate(X_data):
    return np.mean(abs(X_data['count']-X_data['pre']))

In [10]:
train2_dict=train2.groupby(['start_geo_id','end_geo_id','create_date','create_hour'])['id'].count().to_dict()

In [11]:
train2['count']=train2.apply(lambda x:train2_dict[tuple([x['start_geo_id'],x['end_geo_id'],x['create_date'],x['create_hour']])],axis=1)

In [12]:
train2=train2[train2[['start_geo_id','end_geo_id','create_date','create_hour']].duplicated()==False]

In [13]:
test1=test1[['start_geo_id','end_geo_id','create_date','create_hour','id']]

In [14]:
train2=train2[['start_geo_id','end_geo_id','create_date','create_hour','count']]

In [15]:
merged_data=test1.merge(train2,on=['start_geo_id','end_geo_id','create_date'],how='inner')

In [16]:
merged_data.columns=['start_geo_id','end_geo_id','create_date','create_hour_x','id','create_hour_y','ct']

In [17]:
result=dict()
def get_count(x):
    if x['id'] in result.keys():
        return
    value=np.array([-1,-1])
    data=merged_data[merged_data['id']==x['id']]
    idx=data['create_hour_x']>data['create_hour_y']
    wt=1.0/np.exp(abs(data['create_hour_x']-data['create_hour_y']))
    if sum(idx)>0:
        value[0]=sum(data['ct'][idx]*wt[idx])/sum(wt[idx])
    if sum(idx==False)>0:
        value[1]=sum(data['ct'][idx==False]*wt[idx==False])/sum(wt[idx==False])
    w=np.array([0.5,0.5])
    flag=value>0
    pre=sum(value[flag]*w[flag])/sum(w[flag])
    result[x['id']]=pre

In [18]:
merged_data.apply(get_count,axis=1)

0         None
1         None
2         None
3         None
4         None
5         None
6         None
7         None
8         None
9         None
10        None
11        None
12        None
13        None
14        None
15        None
16        None
17        None
18        None
19        None
20        None
21        None
22        None
23        None
24        None
25        None
26        None
27        None
28        None
29        None
          ... 
115043    None
115044    None
115045    None
115046    None
115047    None
115048    None
115049    None
115050    None
115051    None
115052    None
115053    None
115054    None
115055    None
115056    None
115057    None
115058    None
115059    None
115060    None
115061    None
115062    None
115063    None
115064    None
115065    None
115066    None
115067    None
115068    None
115069    None
115070    None
115071    None
115072    None
Length: 115073, dtype: object

In [19]:
X_data['pre']=X_data['id'].apply(lambda x:result[x] if x in result.keys() else 1)

In [20]:
evaluate(X_data)

0.751340753128424

In [21]:
X_data

,id,count,pre
949,0,1,1.0
950,1,1,1.0
955,2,5,2.0
956,3,2,1.0
958,4,4,3.0
959,5,5,1.0
960,6,4,2.5
961,7,1,1.0
962,8,2,1.5
963,9,1,2.0
